In [1]:
import pp
import numpy as np
from pyPoseidon.dem import *
from pyPoseidon.utils.fix import *
import shapely

## Set

In [7]:
dic = {'minlon': 0., # lat/lon window
       'maxlon' : 13.,
       'minlat' : 58.,
       'maxlat' : 66.5,
       'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

In [12]:
## lat,lon grid
resolution=.1
lon=np.arange(dic['minlon'],dic['maxlon'],resolution)
lat=np.arange(dic['minlat'],dic['maxlat'],resolution)
x, y = np.meshgrid(lon,lat)

In [13]:
dic.update({'grid_x':x, 'grid_y':y})

In [14]:
#Define the coastline shapefile
shapefile = '/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_%sm_coastline.shp' % \
                    {'l':110, 'i':50, 'h':10}['i']

In [17]:
dem = gebco(**dic) # get bathymetry


extracting dem from /Users/brey/DATA/GEBCO_2014_2D.nc

dem done


In [19]:
#define coastline
    
shp = gp.GeoDataFrame.from_file(shapefile)

           
xp = dem.Dataset.ilons.values
yp = dem.Dataset.ilats.values
    
    
#put them all in a list
ls=[]
for i in range(shp.shape[0]):
    il = shp.loc[i,'geometry']
    try:
        #print len(il)
        for k in range(len(list(il.geoms))): # for MultiLineStrings
            ls.append(list(il.geoms)[k])
    except:
        ls.append(il)
            
sall = geometry.MultiLineString(ls) # join
c = ops.linemerge(sall) #merge
    
#Select the Line Strings that correspond to our grid
#create a polygon of the grid
grp=geometry.Polygon([(xp.min(),yp.min()),(xp.min(),yp.max()),(xp.max(),yp.max()),(xp.max(),yp.min())])    
    
cl=[] #initialize
#add Polygons if they intersect with the grid
for i in range(len(c)):
    z = geometry.Polygon(c[i])
    if z.intersects(grp): 
        cl.append(c[i])
    
cll = geometry.MultiLineString(cl) #join them into a Multiline
cg = ops.linemerge(cll) #merge parts if possible
    


In [191]:
def loop(xp,yp,cg):
#check if the grid polygons intersect the shoreline
    gps = []
    igps = []
    for i in range(xp.shape[0]-1):
        for j in range(xp.shape[1]-1):
            p = shapely.geometry.Polygon([(xp[i,j],yp[i,j]),(xp[i+1,j],yp[i+1,j]),(xp[i+1,j+1],yp[i+1,j+1]),(xp[i,j+1],yp[i,j+1])])
            if not p.intersects(cg): 
                gps.append(p)
                igps.append([i+1,j+1])
    
    return igps

## PP loop

In [145]:
job_server = pp.Server() 

INFO:pp:Creating server instance (pp-1.6.6)
INFO:pp:Running on Python 2.7.15 darwin
INFO:pp:pp local server started with 8 workers


In [146]:
job_server.get_ncpus()

8

In [147]:
ncpus=4

In [148]:
job_server.set_ncpus(ncpus)

In [149]:
xstep=xp.shape[0]/ncpus
ystep=xp.shape[1]/ncpus
xstep,ystep

(21, 32)

In [150]:
xp.shape

(85, 130)

In [293]:
%%timeit
jobs=[]
for l in range(ncpus):
#    xl=xp[l*xstep:(l+1)*xstep,l*ystep:(l+1)*ystep]
#    yl=yp[l*xstep:(l+1)*xstep,l*ystep:(l+1)*ystep]   
    xl=xp[l*xstep:(l+1)*xstep+1,:]
    yl=yp[l*xstep:(l+1)*xstep+1,:]   


    jobs.append(job_server.submit(loop,(xl,yl,cg,),modules=('shapely',)))

ps=jobs[0]()
for l in range(1,ncpus):    
    ps = np.vstack([ps, [[i+l*xstep,j] for [i,j] in jobs[l]()]])

INFO:pp:Task 32 started
INFO:pp:Task 33 started
INFO:pp:Task 34 started
INFO:pp:Task 35 started
INFO:pp:Task 36 started
INFO:pp:Task 37 started
INFO:pp:Task 38 started
INFO:pp:Task 39 started
INFO:pp:Task 40 started
INFO:pp:Task 41 started
INFO:pp:Task 42 started
INFO:pp:Task 43 started
INFO:pp:Task 44 started
INFO:pp:Task 45 started
INFO:pp:Task 46 started
INFO:pp:Task 47 started


1 loop, best of 3: 4.87 s per loop


## One core

In [285]:
%%timeit 
g1 = loop(xp,yp,cg)

1 loop, best of 3: 17.8 s per loop


In [287]:
solo = np.array(g1)

In [294]:
np.array_equal(solo,ps)

True

## Add extra points from boundary


In [ ]:
for [i,j] in ps:
if i == 0 : 
        igps.append([i,j])
        igps.append([i,j+1])
                if j == 0 : 
                    igps.append([i,j])
                    igps.append([i+1,j])

